In [1]:
# my notebook was corrupted so had add this line
import warnings
warnings.filterwarnings('ignore', category=UserWarning)
#same imports
#vision
import cv2
import mediapipe as mp
import numpy as np
#math and time stuff for detection
import math
from time import time as now
import time as time_module
from collections import deque
# visualization
from PIL import Image
from IPython.display import display, clear_output
from mediapipe.tasks import python
from mediapipe.tasks.python import vision


In [2]:
#unchanged
def euclid(p1, p2):
    return math.hypot(p1.x - p2.x, p1.y - p2.y)

In [3]:
import joblib
clf = joblib.load("models/confusion_tree.joblib")
print("Model loaded:", type(clf))# we take our trained classifer


Model loaded: <class 'sklearn.tree._classes.DecisionTreeClassifier'>


In [4]:
base_options = python.BaseOptions(
    model_asset_path="face_landmarker.task"   # changed because MediaPipe wasn't compatible with Python 3.10
)

options = vision.FaceLandmarkerOptions(
    base_options=base_options,
    running_mode=vision.RunningMode.VIDEO,
    num_faces=1,
    output_face_blendshapes=False
)

detector = vision.FaceLandmarker.create_from_options(options)


In [5]:
#took it from stack overflow
# Gaze & Eye Landmarks (refined landmarks require refine_landmarks=True)
right_iris_center = 474     # Right iris/pupil center
left_iris_center = 468      # Left iris/pupil center  
right_eye_inner = 263       # Right eye inner corner (near nose)
left_eye_inner = 362        # Left eye inner corner (near nose)
right_eye_outer = 133       # Right eye outer corner
left_eye_outer = 33        # Left eye outer corner 
# Brow Landmarks
left_brow_inner = 107       # Left eyebrow inner point  
right_brow_inner = 336      # Right eyebrow inner point
# Face Contour
left_face_contour = 234     # Left jaw/cheek extreme
right_face_contour = 454    # Right jaw/cheek extreme
# Mouth Landmarks
upper_left_lip_corner = 61  # Upper outer lip left corner
lower_right_lip_corner = 291 # Lower outer lip right corner (code pairs with 61)
upper_inner_lip_center = 13 # Top inner lip center
lower_inner_lip_center = 14 # Bottom inner lip center
# Nose
nose_tip = 1                # Nose tip (smile reference)
#extraction of these features
def browratio(lm):
     return euclid(lm[left_brow_inner], lm[right_brow_inner]) / euclid(lm[left_face_contour], lm[right_face_contour])
def mouthfeature(lm):
    left, right = lm[upper_left_lip_corner], lm[lower_right_lip_corner]
    top, bottom = lm[upper_inner_lip_center], lm[lower_inner_lip_center]
    nose = lm[nose_tip]
    facewidth = euclid(lm[left_face_contour], lm[right_face_contour])
    mouth_w = euclid(left, right) / facewidth
    mouth_open = euclid(top, bottom) / facewidth
    smile_up = (nose.y - (left.y + right.y) / 2) / facewidth
    return mouth_w, mouth_open, smile_up
def headroll(lm):
    L, R = lm[left_face_contour], lm[right_face_contour]
    return math.degrees(math.atan2(R.y - L.y, R.x - L.x))

In [6]:
def brow_confusion_rule(landmarks):
    brow_l = landmarks[left_brow_inner]
    brow_r = landmarks[right_brow_inner]
    face_l = landmarks[left_face_contour]
    face_r = landmarks[right_face_contour]
    facewidth = euclid(face_l, face_r)
    if facewidth == 0:
        return False, 0.0
    brow_dist = euclid(brow_l, brow_r)
    ratio = brow_dist / facewidth
    Brow_confusion = 0.1  # adjust this value as needed
    return ratio < Brow_confusion, ratio

In [7]:
#same features from media test1 with added gaze up down etc
def is_happy(self, lm):
    # mouth corners and opening
    left = lm[upper_left_lip_corner ]
    right = lm[lower_right_lip_corner]
    top = lm[upper_inner_lip_center]
    bottom = lm[lower_inner_lip_center]
    face_l = lm[left_face_contour]
    face_r = lm[right_face_contour]
    facewidth = euclid(face_l, face_r)
    if facewidth == 0:
        return False
    mouthwidth = euclid(left, right) / facewidth
    mouthopen = euclid(top, bottom) / facewidth
    smileup = ((left.y + right.y) / 2 - lm[nose_tip].y) / facewidth
    return (
        mouthwidth > 0.33 and
        mouthopen > 0.02 and
        smileup < -0.01
    )
def get_vertical_gaze(self, lm):
    iris = lm[right_iris_center]
    upper = lm[386]
    lower = lm[374]
    eye_height = euclid(upper, lower)
    if eye_height == 0:
        return "CENTER"
    ratio = euclid(iris, upper) / eye_height
    if ratio > 0.35:
        return "UP"
    elif ratio < 0.65:
        return "DOWN"
    return "CENTER"
def get_gaze(self, landmarks):
    # ---- Horizontal ----
    iris = landmarks[474]
    inner = landmarks[263]
    outer = landmarks[362]
    eye_width = euclid(inner, outer)
    gaze_h = "CENTER"
    if eye_width != 0:
        ratio_h = euclid(iris, inner) / eye_width
        if ratio_h < 0.2:
            gaze_h = "RIGHT"
        elif ratio_h > 0.8:
            gaze_h = "LEFT"
    #  Vertical 
    gaze_v = self.get_vertical_gaze(landmarks)

    # ---- Merge ----# right to left
    if gaze_h != "CENTER":
        return gaze_h
    if gaze_v != "CENTER":# up to down
        return gaze_v

    return "CENTER"


In [14]:
import pandas as pd

class StudentIntegrityMonitor:
    """
    Complete monitoring system for student integrity detection.
    Tracks: Gaze direction, Confusion (ML + rule-based), Happiness, Looking away
    """
    def __init__(self, detector, clf, gaze_limit=4.5):
        self.detector = detector
        self.clf = clf
        self.gaze_limit = gaze_limit  # seconds before flagging "looking away"
        self.look_away_st = None  # timestamp when looking away started
        self.pred_buffer = deque(maxlen=7)  # smoothing buffer for ML predictions
        self.frame_count = 0
        self.last_timestamp_ms = 0  # Track last timestamp for monotonicity
    
    # ---- FEATURE EXTRACTION METHODS ----
    def browratio(self, lm):
        """Calculate brow width relative to face width"""
        return euclid(lm[left_brow_inner], lm[right_brow_inner]) / euclid(lm[left_face_contour], lm[right_face_contour])
    
    def mouthfeature(self, lm):
        """Extract mouth width, opening, and smile angle"""
        left, right = lm[upper_left_lip_corner], lm[lower_right_lip_corner]
        top, bottom = lm[upper_inner_lip_center], lm[lower_inner_lip_center]
        nose = lm[nose_tip]
        facewidth = euclid(lm[left_face_contour], lm[right_face_contour])
        mouth_w = euclid(left, right) / facewidth
        mouth_open = euclid(top, bottom) / facewidth
        smile_up = (nose.y - (left.y + right.y) / 2) / facewidth
        return mouth_w, mouth_open, smile_up
    
    def headroll(self, lm):
        """Calculate head roll angle from face contour"""
        L, R = lm[left_face_contour], lm[right_face_contour]
        return math.degrees(math.atan2(R.y - L.y, R.x - L.x))
    
    # ---- DETECTION METHODS ----
    def is_happy(self, lm):
        """Detect if student is happy/excited based on mouth features"""
        left = lm[upper_left_lip_corner]
        right = lm[lower_right_lip_corner]
        top = lm[upper_inner_lip_center]
        bottom = lm[lower_inner_lip_center]
        face_l = lm[left_face_contour]
        face_r = lm[right_face_contour]
        facewidth = euclid(face_l, face_r)
        if facewidth == 0:
            return False
        mouthwidth = euclid(left, right) / facewidth
        mouthopen = euclid(top, bottom) / facewidth
        smileup = ((left.y + right.y) / 2 - lm[nose_tip].y) / facewidth
        return (
        (mouthwidth > 0.33 and
            mouthopen > 0.02) or
            smileup < -0.3
        )
    
    def get_vertical_gaze(self, lm):
        """Detect vertical gaze direction (UP, CENTER, DOWN)"""
        iris = lm[right_iris_center]
        upper = lm[386]
        lower = lm[374]
        eye_height = euclid(upper, lower)
        if eye_height == 0:
            return "CENTER"
        ratio = euclid(iris, upper) / eye_height
        # Small ratio = iris close to upper eyelid = looking UP
        # Large ratio = iris far from upper eyelid = looking DOWN
        if ratio < 0.35:
            return "UP"
        elif ratio > 0.65:
            return "DOWN"
        return "CENTER"
    
    def get_gaze(self, landmarks):
        """Detect horizontal and vertical gaze direction"""
        # ---- Horizontal ----
        iris = landmarks[474]
        inner = landmarks[263]
        outer = landmarks[362]
        eye_width = euclid(inner, outer)
        gaze_h = "CENTER"
        if eye_width != 0:
            ratio_h = euclid(iris, inner) / eye_width
            if ratio_h < 0.2:
                gaze_h = "RIGHT"
            elif ratio_h > 0.8:
                gaze_h = "LEFT"
        # ---- Vertical ----
        gaze_v = self.get_vertical_gaze(landmarks)
        
        # ---- Merge ----
        if gaze_h != "CENTER":
            return gaze_h
        if gaze_v != "CENTER":
            return gaze_v
        return "CENTER"
    
    def brow_confusion_rule(self, landmarks):
        """Rule-based confusion detection using brow position"""
        brow_l = landmarks[left_brow_inner]
        brow_r = landmarks[right_brow_inner]
        face_l = landmarks[left_face_contour]
        face_r = landmarks[right_face_contour]
        facewidth = euclid(face_l, face_r)
        if facewidth == 0:
            return False, 0.0
        brow_dist = euclid(brow_l, brow_r)
        ratio = brow_dist / facewidth
        Brow_confusion = 0.1  # threshold
        return ratio < Brow_confusion, ratio
    
    def predict(self, lm):
        """ML prediction for confusion"""
        brow = self.browratio(lm)
        mouth_w, mouth_open, smile_up = self.mouthfeature(lm)
        head_roll_abs = abs(self.headroll(lm))
        X = pd.DataFrame(
            [[brow, mouth_w, mouth_open, smile_up, head_roll_abs]],
            columns=self.clf.feature_names_in_
        )
        pred = self.clf.predict(X)[0]  # 1=>confused, 0=>not confused
        return pred, {
            "brow": brow,
            "smile_up": smile_up,
            "roll": head_roll_abs
        }
    
    def smooth_prediction(self, pred):
        """Smooth ML predictions using majority voting over 7 frames"""
        self.pred_buffer.append(pred)
        return sum(self.pred_buffer) >= (len(self.pred_buffer) // 2 + 1)
    
    # ---- MAIN PIPELINE ----
    def process(self, frame):
        """
        Complete analysis pipeline:
        1. Extract face landmarks
        2. Check gaze direction & timeout
        3. Check confusion (ML + rule-based)
        4. Check happy state
        5. Return final status with features
        """
        self.frame_count += 1
        # Use actual time-based timestamp to ensure monotonicity
        current_time_ms = int(time_module.time() * 1000)
        # Ensure timestamp is always strictly increasing
        if current_time_ms <= self.last_timestamp_ms:
            timestamp_ms = self.last_timestamp_ms + 1
        else:
            timestamp_ms = current_time_ms
        self.last_timestamp_ms = timestamp_ms
        
        rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        mp_image = mp.Image(
            image_format=mp.ImageFormat.SRGB,
            data=rgb
        )
        result = self.detector.detect_for_video(mp_image, timestamp_ms)
        
        gaze = "CENTER"
        feats = {}
        
        # ---- INTEGRITY CHECKS ----
        if not result.face_landmarks:
            return "No Face", "No Face", gaze, feats
        if len(result.face_landmarks) > 1:
            return "Multiple Faces", "Alert", gaze, feats
        
        landmarks = result.face_landmarks[0]
        
        # ---- GAZE (INTEGRITY + TIME-BASED) ----
        gaze = self.get_gaze(landmarks)
        if gaze != "CENTER":
            if self.look_away_st is None:
                self.look_away_st = now()
            elif now() - self.look_away_st > self.gaze_limit:
                return "Looking Away", "Alert", gaze, feats
        else:
            self.look_away_st = None
        
        # ---- CONFUSION (ML FIRST + SMOOTHING) ----
        pred, feats = self.predict(landmarks)
        confused = self.smooth_prediction(pred)
        if confused:
            return "Confused", "Confused", gaze, feats
        
        # ---- CONFUSION FALLBACK (RULE-BASED SAFETY NET) ----
        rule_confused, ratio = self.brow_confusion_rule(landmarks)
        if rule_confused:
            return "Confused", "Confused", gaze, feats
        
        # ---- HAPPY (RULE-BASED OVERRIDE) ----
        if self.is_happy(landmarks):
            return "Happy", "Happy", gaze, feats
        
        # ---- DEFAULT ----
        return "Focused", "Focused", gaze, feats

In [15]:
def process(self, frame):
    rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    mp_image = mp.Image(
        image_format=mp.ImageFormat.SRGB,
        data=rgb
    )
    result = self.detector.detect(mp_image)
    # same Thing
    status = "No Face"
    state = "Neutral"
    gaze = "Center"
    #  INTEGRITY CHECKS 
    if not result.face_landmarks:
        return "No Face", "No Face", gaze, {}
    if len(result.face_landmarks) > 1:
        return "Multiple Faces", "Alert", gaze, {}
    landmarks = result.face_landmarks[0]
    #  GAZE (INTEGRITY, TIME-BASED) 
    gaze = self.get_gaze(landmarks)
    if gaze != "Center":
        if self.look_away_st is None:
            self.look_away_st = now()
        elif now() - self.look_away_st > self.gaze_limit:
            return "Looking Away", "Alert", gaze, {}
    else:
        self.look_away_st = None
    #  CONFUSION (ML FIRST) 
    pred, feats = predict(landmarks)
    if pred == 1:
        return "Confused", "Confused", gaze, feats

    # ---- CONFUSION FALLBACK (RULE-BASED SAFETY NET) ----
    rule_confused, ratio = brow_confusion_rule(landmarks)
    if rule_confused:
        return "Confused", "Confused", gaze, feats
    #  HAPPY (RULE-BASED OVERRIDE) 
    if self.is_happy(landmarks):
        return "Happy", "Happy", gaze, feats
    #  DEFAULT 
    return "Focused", "Focused", gaze, feats

In [16]:
def predict(lm):
    brow = browratio(lm)
    mouth_w, mouth_open, smile_up = mouthfeature(lm)
    head_roll_abs = abs(headroll(lm))
    X = pd.DataFrame(
        [[brow, mouth_w, mouth_open, smile_up, head_roll_abs]],
        columns=clf.feature_names_in_
    )
    pred = clf.predict(X)[0]  # 1 => confused, 0 => nope
    return pred, {
        "brow": brow,
        "smile": smile_up,
        "roll": head_roll_abs
    }


In [17]:
import pandas as pd

def predict(lm):
    brow = browratio(lm)
    mouth_w, mouth_open, smile_up = mouthfeature(lm)
    head_roll_abs = abs(headroll(lm))
    X = pd.DataFrame(
        [[brow, mouth_w, mouth_open, smile_up, head_roll_abs]],
        columns=clf.feature_names_in_
    )
    pred = clf.predict(X)[0]  # 1=>confused, 0=>nope
    return pred, {
        "brow": brow,
        "smile_up": smile_up,
        "roll": head_roll_abs
    }

In [18]:
pred_buffer = deque(maxlen=7)
def smooth_prediction(pred):
    pred_buffer.append(pred)
    return sum(pred_buffer) >= (len(pred_buffer)//2 + 1)

In [19]:
import time

# Instantiate the monitor with your detector and model
monitor = StudentIntegrityMonitor(
    detector=detector,
    clf=clf,
    gaze_limit=4.5  # Flag "looking away" after 3 seconds
)

cap = cv2.VideoCapture(0)
print("Starting live verification. Press Q to quit.")
print("\nDetecting: Focused, Confused, Happy, Looking Away, No Face, Multiple Faces")

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Run complete pipeline
    status, state, gaze, feats = monitor.process(frame)
    
    # Color mapping
    color_map = {
        "Focused": (255, 255, 0),      # Yellow
        "Confused": (0, 0, 255),       # Red
        "Happy": (0, 255, 0),          # Green
        "Looking Away": (0, 165, 255), # Orange
        "No Face": (128, 128, 128),    # Gray
        "Multiple Faces": (0, 255, 255) # Cyan
    }
    color = color_map.get(status, (128, 128, 128))
    
    # Display status and gaze
    cv2.putText(frame, status, (20, 40),
                cv2.FONT_HERSHEY_SIMPLEX, 1.2, color, 3)
    cv2.putText(frame, f"Gaze: {gaze}", (20, 80),
                cv2.FONT_HERSHEY_SIMPLEX, 0.8, (200, 200, 200), 2)
    
    # Display extracted features
    y_offset = 120
    if feats:
        cv2.putText(frame, f"Brow: {feats.get('brow', 0):.3f}", (20, y_offset),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 200, 0), 2)
        cv2.putText(frame, f"Smile: {feats.get('smile_up', 0):.3f}", (20, y_offset + 30),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 200, 0), 2)
        cv2.putText(frame, f"Roll: {feats.get('roll', 0):.3f}", (20, y_offset + 60),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 200, 0), 2)

    cv2.imshow("Student Integrity Monitor", frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()
print("\nMonitoring stopped.")


Starting live verification. Press Q to quit.

Detecting: Focused, Confused, Happy, Looking Away, No Face, Multiple Faces

Monitoring stopped.


In [15]:
clf.feature_names_in_


array(['brow_ratio', 'smile_up', 'mouth_open', 'mouth_width',
       'head_roll_abs'], dtype=object)